# Script to create composites of SST gradient based on ENSO phase

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from cartopy import crs as ccrs
import xarray as xr

In [3]:
sstlink = '/Users/noahrosenberg/Downloads/era5_sst_1940_2022_1deg.nc'

sst_test = xr.load_dataset(f'{sstlink}',engine='netcdf4')

In [4]:
from compute_gulfstream_gradient import get_gs_coords_alltime,get_total_gradient

FileNotFoundError: [Errno 2] No such file or directory: b'/Users/gliu/Dropbox (MIT)/Glenn Liu\xe2\x80\x99s files/Home/Work_Portable/ICTP/Project/Data/ERA5_sst_test.nc'